In [1]:
# =========================
# Library
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
import Levenshtein
import difflib
from contextlib import contextmanager
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import log_loss
import logging
warnings.filterwarnings('ignore')
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# =========================
# Constant
# =========================
TRAIN_PATH = "../data/train.csv"
TARGET = "point_of_interest"

In [3]:
# =========================
# Settings
# =========================
exp = "075"
val_fold = 0
if not os.path.exists(f"../output/exp/ex{exp}"):
    os.makedirs(f"../output/exp/ex{exp}")
    os.makedirs(f"../output/exp/ex{exp}/model")
LOGGER_PATH = f"../output/exp/ex{exp}/ex_{exp}.txt"
MODEL_PATH = f"../output/exp/ex{exp}/model"

SEED = 0
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'binary',
               'max_depth': -1,
               'learning_rate': 0.1,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"binary_logloss",
              'num_threads':56
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 4000
}

fe_list = ["../output/fe/fe066.pkl",
           "../output/fe/fe067.pkl",
           "../output/fe/fe068.pkl",
           "../output/fe/fe069.pkl"]

In [4]:
# =========================
# Functions
# =========================

def calc_loss(y_true, y_pred):
    return  log_loss(y_true, y_pred)


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [5]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2022-06-07 11:38:19,784 - INFO - logger set up


<RootLogger root (DEBUG)>

In [6]:
train = pd.read_pickle(fe_list[0]) 

In [7]:
for i in range(1,len(fe_list)):
    _train = pd.read_pickle(fe_list[i])
    train = pd.concat([train,_train],axis=1)
    del _train
    gc.collect()
# train["target"] = train[TARGET] == train["near_target"]
# train["target"] = train["target"].astype(int)

In [8]:
drop_cols = ['id', 'name','match','oof', 'oof_rank',
       'address', 'city', 'state', 'zip','country', 'url', 'phone',
       'categories', 'point_of_interest', 'set',
       'near_target', 'near_id', 'near_name', 'near_address', 'near_city',
       'near_state', 'near_zip', 'near_country', 'near_url', 'near_phone',
       'near_categories','target']
features = [i for i in train.columns if i not in drop_cols]

In [9]:
features

['latitude',
 'longitude',
 'rank',
 'd_near',
 'near_latitude',
 'near_longitude',
 'name_gesh',
 'name_leven',
 'name_jaro',
 'address_gesh',
 'address_leven',
 'address_jaro',
 'city_gesh',
 'city_leven',
 'city_jaro',
 'state_gesh',
 'state_leven',
 'state_jaro',
 'zip_gesh',
 'zip_leven',
 'zip_jaro',
 'url_gesh',
 'url_leven',
 'url_jaro',
 'phone_gesh',
 'phone_leven',
 'phone_jaro',
 'categories_gesh',
 'categories_leven',
 'categories_jaro',
 'distance',
 'distance_rank',
 'name_gesh_mean',
 'name_gesh_max',
 'near_name_gesh_mean',
 'near_name_gesh_max',
 'name_gesh_mean_rate',
 'name_gesh_max_rate',
 'near_name_gesh_mean_rate',
 'near_name_gesh_max_rate',
 'name_leven_mean',
 'name_leven_min',
 'near_name_leven_mean',
 'near_name_leven_min',
 'name_leven_mean_rate',
 'name_leven_min_rate',
 'near_name_leven_mean_rate',
 'near_name_leven_min_rate',
 'name_jaro_mean',
 'name_jaro_max',
 'near_name_jaro_mean',
 'near_name_jaro_max',
 'name_jaro_mean_rate',
 'name_jaro_max_rate',

In [10]:
with timer("lightgbm"):
    drop_cols = []
    feature_importances = pd.DataFrame()
    categorical_features = []
    y_oof = np.empty([len(train),])
    fold = 0
    with timer(f"fold {fold}"):
        x_train = train[features].values.astype(np.float32)
        y = train["target"].values
        y_pred_valid, y_pred_test, valid_loss, importances, best_iter, model = train_lgbm(
                    x_train, y ,None, None,None,
                    categorical_features=categorical_features,
                    feature_name=features,
                    fold_id=fold,
                    lgb_params=LGBM_PARAMS,
                    fit_params=LGBM_FIT_PARAMS,
                    loss_func=calc_loss,
                    calc_importances=True
                )
        save_path = f"{MODEL_PATH}/lgb_fold{fold}.pkl"
        pickle.dump(model, open(save_path, 'wb'))
        model.save_model(f"{MODEL_PATH}/lgb_fold{fold}.txt")

2022-06-07 12:35:08,135 - INFO - [fold 0] done in 3207 s
2022-06-07 12:35:08,138 - INFO - [lightgbm] done in 3207 s
